In [ ]:
# -------------------------------
# 1. INSTALL & IMPORT DEPENDENCIES
# -------------------------------
!pip install -q kaggle
!pip install -q scikit-learn

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# -------------------------------
# 2. UPLOAD KAGGLE.JSON
# -------------------------------
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# -------------------------------
# 3. DOWNLOAD & UNZIP DATASET
# -------------------------------
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection
!unzip -q brain-mri-images-for-brain-tumor-detection.zip -d data

# -------------------------------
# 4. ORGANIZE DATASET (Train/Val/Test Split)
# -------------------------------
import shutil
from sklearn.model_selection import train_test_split

def organize_dataset(base_path='data/brain_tumor_dataset'):
    os.makedirs('dataset/train/yes', exist_ok=True)
    os.makedirs('dataset/train/no', exist_ok=True)
    os.makedirs('dataset/val/yes', exist_ok=True)
    os.makedirs('dataset/val/no', exist_ok=True)
    os.makedirs('dataset/test/yes', exist_ok=True)
    os.makedirs('dataset/test/no', exist_ok=True)

    yes_images = os.listdir(f'{base_path}/yes')
    no_images = os.listdir(f'{base_path}/no')

    yes_train, yes_test = train_test_split(yes_images, test_size=0.2, random_state=42)
    yes_train, yes_val = train_test_split(yes_train, test_size=0.2, random_state=42)

    no_train, no_test = train_test_split(no_images, test_size=0.2, random_state=42)
    no_train, no_val = train_test_split(no_train, test_size=0.2, random_state=42)

    for name, group in [('train', yes_train), ('val', yes_val), ('test', yes_test)]:
        for img in group:
            shutil.copy(f'{base_path}/yes/{img}', f'dataset/{name}/yes/{img}')

    for name, group in [('train', no_train), ('val', no_val), ('test', no_test)]:
        for img in group:
            shutil.copy(f'{base_path}/no/{img}', f'dataset/{name}/no/{img}')

organize_dataset()

# -------------------------------
# 5. DATA LOADING & AUGMENTATION
# -------------------------------
IMG_SIZE = 224
BATCH_SIZE = 32

train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

val_aug = ImageDataGenerator(rescale=1./255)

train_data = train_aug.flow_from_directory(
    "dataset/train", target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode="binary"
)

val_data = val_aug.flow_from_directory(
    "dataset/val", target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode="binary"
)

test_data = val_aug.flow_from_directory(
    "dataset/test", target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode="binary", shuffle=False
)

# -------------------------------
# 6. CALCULATE CLASS WEIGHTS
# -------------------------------
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_data.classes),
                                     y=train_data.classes)
class_weights = dict(enumerate(class_weights))

# -------------------------------
# 7. MODEL DEFINITION (FINE-TUNED VGG16)
# -------------------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
for layer in base_model.layers[:-4]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# -------------------------------
# 8. TRAINING
# -------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=100,
    class_weight=class_weights,
    callbacks=[early_stop, lr_reduce]
)

# -------------------------------
# 9. PLOT LOSS & ACCURACY
# -------------------------------
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# -------------------------------
# 10. EVALUATE ON TEST SET
# -------------------------------
pred_probs = model.predict(test_data)
y_pred = (pred_probs > 0.5).astype('int32').reshape(-1)
y_true = test_data.classes

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=["No Tumor", "Tumor"]))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["No Tumor", "Tumor"], yticklabels=["No Tumor", "Tumor"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()




Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
brain-mri-images-for-brain-tumor-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data/brain_tumor_dataset/no/1 no.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Found 161 images belonging to 2 classes.
Found 41 images belonging to 2 classes.
Found 51 images belonging to 2 classes.
Model: "functional_2"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
 Total params: 14,780,481 (56.38 MB)
 Trainable params: 7,145,217 (27.26 MB)
 Non-trainable params: 7,635,264 (29.13 MB)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4509 - loss: 0.7831 - val_accuracy: 0.6098 - val_loss: 0.6704 - learning_rate: 1.0000e-04
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 415ms/step - accuracy: 0.6534 - loss: 0.6701 - val_accuracy: 0.6098 - val_loss: 0.6277 - learning_rate: 1.0000e-04
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 516ms/step - accuracy: 0.5827 - loss: 0.6775 - val_accuracy: 0.6585 - val_loss: 0.6088 - learning_rate: 1.0000e-04
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 508ms/step - accuracy: 0.5545 - loss: 0.6538 - val_accuracy: 0.6098 - val_loss: 0.5872 - learning_rate: 1.0000e-04
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 538ms/step - accuracy: 0.6595 - loss: 0.6446 - val_accuracy: 0.6585 - val_loss: 0.5558 - learning_rate: 1.0000e-04
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 407ms/step - accuracy: 0.7065 - loss: 0.5829 - val_accuracy: 0.7073 - val_loss: 0.4886 - learning_rate: 1.0000e-04
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 518ms/step - accuracy: 0.5272 - loss: 0.7518 - val_accuracy: 0.8049 - val_loss: 0.4778 - learning_rate: 1.0000e-04
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 437ms/step - accuracy: 0.7251 - loss: 0.5797 - val_accuracy: 0.7073 - val_loss: 0.4746 - learning_rate: 1.0000e-04
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 541ms/step - accuracy: 0.7236 - loss: 0.5079 - val_accuracy: 0.7561 - val_loss: 0.4745 - learning_rate: 1.0000e-04
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 398ms/step - accuracy: 0.7918 - loss: 0.4842 - val_accuracy: 0.7073 - val_loss: 0.5598 - learning_rate: 1.0000e-04
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 409ms/step - accuracy: 0.8081 - loss: 0.5058 - val_accuracy: 0.7073 - val_loss: 0.5435 - learning_rate: 1.0000e-04
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 509ms/step - accuracy: 0.5501 - loss: 0.6345 - val_accuracy: 0.8537 - val_loss: 0.4082 - learning_rate: 1.0000e-04
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 658ms/step - accuracy: 0.8758 - loss: 0.3764 - val_accuracy: 0.7317 - val_loss: 0.5097 - learning_rate: 1.0000e-04
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 405ms/step - accuracy: 0.8577 - loss: 0.4080 - val_accuracy: 0.7561 - val_loss: 0.4237 - learning_rate: 1.0000e-04
Epoch 15/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 483ms/step - accuracy: 0.7907 - loss: 0.4484 - val_accuracy: 0.8537 - val_loss: 0.3999 - learning_rate: 1.0000e-04
Epoch 16/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 505ms/step - accuracy: 0.6996 - loss: 0.6402 - val_accuracy: 0.7561 - val_loss: 0.5330 - learning_rate: 1.0000e-04
Epoch 17/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 514ms/step - accuracy: 0.8633 - loss: 0.4273 - val_accuracy: 0.7561 - val_loss: 0.4576 - learning_rate: 1.0000e-04
Epoch 18/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 395ms/step - accuracy: 0.7863 - loss: 0.4189 - val_accuracy: 0.8780 - val_loss: 0.3571 - learning_rate: 1.0000e-04
Epoch 19/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 423ms/step - accuracy: 0.7973 - loss: 0.4370 - val_accuracy: 0.7805 - val_loss: 0.4598 - learning_rate: 1.0000e-04
Epoch 20/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 394ms/step - accuracy: 0.8077 - loss: 0.5121 - val_accuracy: 0.8537 - val_loss: 0.3892 - learning_rate: 1.0000e-04
Epoch 21/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 552ms/step - accuracy: 0.7943 - loss: 0.3474 - val_accuracy: 0.8780 - val_loss: 0.3137 - learning_rate: 1.0000e-04
Epoch 22/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 400ms/step - accuracy: 0.8783 - loss: 0.3756 - val_accuracy: 0.8537 - val_loss: 0.3146 - learning_rate: 1.0000e-04
Epoch 23/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 408ms/step - accuracy: 0.8526 - loss: 0.3361 - val_accuracy: 0.8537 - val_loss: 0.3245 - learning_rate: 1.0000e-04
Epoch 24/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 587ms/step - accuracy: 0.9138 - loss: 0.2222 - val_accuracy: 0.9024 - val_loss: 0.2630 - learning_rate: 1.0000e-04
Epoch 25/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 419ms/step - accuracy: 0.8961 - loss: 0.3784 - val_accuracy: 0.9024 - val_loss: 0.2596 - learning_rate: 1.0000e-04
Epoch 26/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 406ms/step - accuracy: 0.9302 - loss: 0.2448 - val_accuracy: 0.8537 - val_loss: 0.3676 - learning_rate: 1.0000e-04
Epoch 27/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 404ms/step - accuracy: 0.8994 - loss: 0.2668 - val_accuracy: 0.8537 - val_loss: 0.3669 - learning_rate: 1.0000e-04
Epoch 28/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 414ms/step - accuracy: 0.7190 - loss: 0.5663 - val_accuracy: 0.6341 - val_loss: 0.8262 - learning_rate: 1.0000e-04
Epoch 29/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 498ms/step - accuracy: 0.6155 - loss: 0.5846 - val_accuracy: 0.8537 - val_loss: 0.3734 - learning_rate: 5.0000e-05
Epoch 30/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 401ms/step - accuracy: 0.8710 - loss: 0.3549 - val_accuracy: 0.9024 - val_loss: 0.3228 - learning_rate: 5.0000e-05

WARNING:tensorflow:5 out of the last 5 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7c0405f6d120> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7c0405f6d120> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
Classification Report:

              precision    recall  f1-score   support

    No Tumor       0.83      0.75      0.79        20
       Tumor       0.85      0.90      0.88        31

    accuracy                           0.84        51
   macro avg       0.84      0.83      0.83        51
weighted avg       0.84      0.84      0.84        51

